In [12]:
%matplotlib inline
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import math
from numpy import array
import os
import time 

In [13]:
class Control():        
    def __init__(self,original):        
        self.orisg = original          # Original signal
        self.consg = np.zeros(original.shape)  # Signal after control

    def datacontrol(self,kpidp): 
        slbeimf = self.orisg[0:12500].copy()
        nor_value = []
        while len(nor_value) <len(self.orisg):
            nor_value = np.concatenate([nor_value, slbeimf])
            
        cimf = []
        for batch in range(len(self.orisg)):
            erro = nor_value[batch]-self.orisg[batch]
            a = (kpidp[0]*erro + kpidp[1]*erro+kpidp[2]*erro)
            erro1 = nor_value[batch]-a
            b = (kpidp[0]*erro1 + kpidp[1]*(erro+erro1)+kpidp[2]*(erro1-erro))
            cimf.append(b)

        eudis =  np.sqrt(np.square(nor_value - cimf))
        return np.array(cimf),eudis      
      
def datasave(path,samplek,record):
    Data = np.concatenate((samplek.reshape(-1,1),record.reshape(-1,1)),axis = 1)
    test = pd.DataFrame(columns=['time','value'],data=Data)
    test.to_csv(path,index = False)

In [5]:
all_imfs = {}
org_sig = np.zeros(87500)

for i in range(1,11):
    filepath = "../../datasources/validation/vmd/IMF{}.CSV".format(i)
    df = pd.read_csv(filepath)

    sample = df['time'].values.copy()
    all_imfs[i] = df['value'].values.copy()
    org_sig = org_sig + np.array(all_imfs[i])

In [6]:
SOA    =  [0.37506667 , 0.29499 , -0.16124236]
NN_old =  [-0.2941,  0.4289,  0.2662]
PSO    =  [0.11257014, 0.20711179, 0.00347529] 
NN_new =  [ 0.5916,  0.3169, -0.2104]

cmode = {0:SOA,1:NN_old,2:PSO,3:NN_new}
cpath = {0:'soa',1:'nn_old',2:'pso',3:'nn_new'}
consg = {}
edu = {}

In [11]:
for i in range(4):
    onvmd = Control(org_sig)
    consg[i],edu[i]= onvmd.datacontrol(cmode[i])

# cata1 = '../result/nonvmd'
# isExists=os.path.exists(cata1)
# if not isExists:
#     os.makedirs(cata1) 
for i in range(4):        
    path1 = '../Data/{}.csv'.format(cpath[i])
    
    datasave(path1,sample,consg[i])
    
    path2 = '../Euclidean distance/Euclidean_{}.csv'.format(cpath[i])
    datasave(path2,sample,edu[i])
    
path3 = '../../datasources/validation/original.csv'
datasave(path3,sample,org_sig)

In [10]:
class lagtest():        
    def __init__(self,original):        
        self.orisg = original          # Original signal
        self.consg = np.zeros(original.shape)  # Signal after control

    def datacontrol(self,kpidp): 
        nor_value = self.orisg[0:125].copy()        
        cimf = []
        for batch in range(len(self.orisg)):
            erro = nor_value[batch]-self.orisg[batch]
            a = (kpidp[0]*erro + kpidp[1]*erro+kpidp[2]*erro)
            erro1 = nor_value[batch]-a
            b = (kpidp[0]*erro1 + kpidp[1]*(erro+erro1)+kpidp[2]*(erro1-erro))
            cimf.append(b)

        eudis =  np.sqrt(np.square(nor_value - cimf))
        return np.array(cimf),eudis      

In [31]:
consg1 = {}
lag = {}

for i in range(4):
    start1 = time.time() 
    ccl = lagtest(org_sig[25000:25125])
    consg1[i]= ccl.datacontrol(cmode[i])
    end1 = time.time()
    lag[i] =  end1 - start1 + 0.01

In [32]:
lag

{0: 0.01, 1: 0.01099945068359375, 2: 0.01, 3: 0.01}